In [36]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.optimizers import Adam

In [20]:
# Load the data
data = pd.read_csv('data/atlantic_post2003.csv')

In [24]:
data['DateTime'] = pd.to_datetime(data['DateTime'])
data.sort_values(by='DateTime', inplace=True)

In [25]:
features = data[['Maximum Wind', 'Minimum Pressure']]
targets = data[['Latitude', 'Longitude']]

In [26]:
def create_sequences(input_data, output_data, sequence_length):
    xs = []
    ys = []
    for i in range(len(input_data) - sequence_length):
        x = input_data[i:(i + sequence_length)]
        y = output_data[i + sequence_length]
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

In [29]:
scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(features)

In [32]:
sequence_length = 5
X, y = create_sequences(scaled_features, targets.values, sequence_length)

In [33]:
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

In [42]:
# LSTM model
model = Sequential([
    
    LSTM(256, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True),
    LSTM(128),
    Dense(2)
])

In [43]:
model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

In [44]:
history = model.fit(X_train, y_train, epochs=100, batch_size=128, validation_split=0.1, verbose=1)

Epoch 1/100
34/34 [==============================] - 5s 58ms/step - loss: 2004.3271 - val_loss: 1388.5474
Epoch 2/100
34/34 [==============================] - 1s 35ms/step - loss: 1341.1296 - val_loss: 1104.5587
Epoch 3/100
34/34 [==============================] - 1s 35ms/step - loss: 1092.7722 - val_loss: 898.1549
Epoch 4/100
34/34 [==============================] - 1s 35ms/step - loss: 907.6383 - val_loss: 742.4294
Epoch 5/100
34/34 [==============================] - 1s 35ms/step - loss: 765.9288 - val_loss: 622.0735
Epoch 6/100
34/34 [==============================] - 1s 34ms/step - loss: 655.5854 - val_loss: 527.5349
Epoch 7/100
34/34 [==============================] - 1s 35ms/step - loss: 568.4775 - val_loss: 453.5933
Epoch 8/100
34/34 [==============================] - 1s 34ms/step - loss: 500.0466 - val_loss: 395.1325
Epoch 9/100
34/34 [==============================] - 1s 34ms/step - loss: 445.9988 - val_loss: 349.6332
Epoch 10/100
34/34 [==============================] - 1s 35

In [41]:
loss = model.evaluate(X_test, y_test, verbose=0)
print("Test loss:", loss)

Test loss: 320.52044677734375
